In [1]:
# Setup Dash
from dash import Dash, dcc, html, dash_table, Input, Output, State
import dash_leaflet as dl
import plotly.express as px
import base64

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Import the CRUD Python module
from animal_shelter import AnimalShelter

###########################
# Data Manipulation / Model
###########################

username = "aacuser"
password = "1234"

# Connect to database via CRUD Module
db = AnimalShelter(username, password)

# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(db.read({}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invalid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will return a new dataframe that does not contain the dropped column(s)
df.drop(columns=['_id'], inplace=True)


# Clean up null values and ensure proper data types
df = df.dropna(subset=['age_upon_outcome_in_weeks', 'location_lat', 'location_long'])
df['age_upon_outcome_in_weeks'] = pd.to_numeric(df['age_upon_outcome_in_weeks'], errors='coerce')
df = df.dropna(subset=['age_upon_outcome_in_weeks'])

print(f"Loaded {len(df)} dog records from database")

###########################
# Query Functions for Filters
###########################

def get_water_rescue_dogs():
    """Query for Water Rescue dogs based on Grazioso Salvare criteria"""
    query = {
        'animal_type': {'$regex': 'Dog', '$options': 'i'},
        'sex_upon_outcome': {'$regex': 'Intact Female', '$options': 'i'},
        'age_upon_outcome_in_weeks': {'$gte': 26, '$lte': 156},
        '$or': [
            {'breed': {'$regex': 'Labrador Retriever', '$options': 'i'}},
            {'breed': {'$regex': 'Chesapeake Bay Retriever', '$options': 'i'}},
            {'breed': {'$regex': 'Newfoundland', '$options': 'i'}}
        ]
    }
    return pd.DataFrame.from_records(db.read(query))

def get_mountain_rescue_dogs():
    """Query for Mountain/Wilderness Rescue dogs"""
    query = {
        'animal_type': {'$regex': 'Dog', '$options': 'i'},
        'sex_upon_outcome': {'$regex': 'Intact Male', '$options': 'i'},
        'age_upon_outcome_in_weeks': {'$gte': 26, '$lte': 156},
        '$or': [
            {'breed': {'$regex': 'German Shepherd', '$options': 'i'}},
            {'breed': {'$regex': 'Alaskan Malamute', '$options': 'i'}},
            {'breed': {'$regex': 'Old English Sheepdog', '$options': 'i'}},
            {'breed': {'$regex': 'Siberian Husky', '$options': 'i'}},
            {'breed': {'$regex': 'Rottweiler', '$options': 'i'}}
        ]
    }
    return pd.DataFrame.from_records(db.read(query))

def get_disaster_rescue_dogs():
    """Query for Disaster/Individual Tracking dogs"""
    query = {
        'animal_type': {'$regex': 'Dog', '$options': 'i'},
        'sex_upon_outcome': {'$regex': 'Intact Male', '$options': 'i'},
        'age_upon_outcome_in_weeks': {'$gte': 20, '$lte': 300},
        '$or': [
            {'breed': {'$regex': 'Doberman Pinscher', '$options': 'i'}},
            {'breed': {'$regex': 'German Shepherd', '$options': 'i'}},
            {'breed': {'$regex': 'Golden Retriever', '$options': 'i'}},
            {'breed': {'$regex': 'Bloodhound', '$options': 'i'}},
            {'breed': {'$regex': 'Rottweiler', '$options': 'i'}}
        ]
    }
    return pd.DataFrame.from_records(db.read(query))

#########################
# Dashboard Layout / View
#########################
app = Dash(__name__)

# Load and encode the Grazioso Salvare logo
try:
    image_filename = 'grazioso_logo.png'  # Update with actual logo filename
    encoded_image = base64.b64encode(open(image_filename, 'rb').read())
    logo_component = html.Img(
        src='data:image/png;base64,{}'.format(encoded_image.decode()),
        style={'height': '80px', 'margin': '10px'}
    )
except:
    # Fallback if logo file is not found
    logo_component = html.Div([
        html.H3("GRAZIOSO SALVARE", style={'color': '#d32f2f', 'margin': '10px'}),
        html.P("Animal Rescue Dashboard", style={'margin': '10px'})
    ])

app.layout = html.Div([
    # Header with logo and title
    html.Div([
        html.A([logo_component], href="https://www.snhu.edu", target="_blank"),
        html.Center(html.B(html.H1('CS-340 Dashboard'))),
        html.Center(html.P('Created by: Eriko Maledhi', 
                          style={'fontStyle': 'italic', 'color': '#666'}))
    ]),
    
    html.Hr(),
    
    # Interactive Filter Options
    html.Div([
        html.H3("Filter Options", style={'textAlign': 'center'}),
        dcc.RadioItems(
            id='filter-type',
            options=[
                {'label': 'Reset (Show All Dogs)', 'value': 'reset'},
                {'label': 'Water Rescue', 'value': 'water'},
                {'label': 'Mountain or Wilderness Rescue', 'value': 'mountain'},
                {'label': 'Disaster or Individual Tracking', 'value': 'disaster'}
            ],
            value='reset',
            labelStyle={'display': 'inline-block', 'margin': '10px'},
            style={'textAlign': 'center', 'margin': '20px'}
        )
    ]),
    
    html.Hr(),
    
    # Data Table
    html.Div([
        html.H3("Animal Data", style={'textAlign': 'center'}),
        dash_table.DataTable(
            id='datatable-id',
            columns=[
                {"name": i, "id": i, "deletable": False, "selectable": True, "type": "numeric" if i == "rec_num" else "text"} 
                for i in df.columns
            ],
            data=df.to_dict('records'),
            page_size=15,
            style_table={'overflowX': 'auto'},
            style_cell={
                'textAlign': 'left',
                'minWidth': '100px',
                'maxWidth': '180px',
                'whiteSpace': 'normal'
            },
            style_data={'whiteSpace': 'normal', 'height': 'auto'},
            sort_action='native',
            filter_action='native',
            filter_options={'case': 'insensitive'},
            row_selectable='single',
            selected_rows=[0],
            style_header={
                'backgroundColor': 'rgb(210, 210, 210)',
                'color': 'black',
                'fontWeight': 'bold'
            }
        )
    ]),
    
    html.Br(),
    html.Hr(),
    
    # Charts Section
    html.Div(className='row', style={'display': 'flex'}, children=[
        html.Div([
            html.H4("Breed Distribution", style={'textAlign': 'center'}),
            html.Div(id='graph-id')
        ], className='col s12 m6', style={'width': '50%', 'padding': '10px'}),
        
        html.Div([
            html.H4("Geolocation Map", style={'textAlign': 'center'}),
            html.Div(id='map-id')
        ], className='col s12 m6', style={'width': '50%', 'padding': '10px'})
    ])
])

#############################################
# Interaction Between Components / Controller
#############################################

@app.callback(
    Output('datatable-id', 'data'),
    [Input('filter-type', 'value')]
)
def update_dashboard(filter_type):
    """Update the data table based on the selected filter"""
    try:
        if filter_type == 'water':
            filtered_df = get_water_rescue_dogs()
        elif filter_type == 'mountain':
            filtered_df = get_mountain_rescue_dogs()
        elif filter_type == 'disaster':
            filtered_df = get_disaster_rescue_dogs()
        else:  # reset
            filtered_df = df
        
        # Clean up the filtered dataframe
        if '_id' in filtered_df.columns:
            filtered_df = filtered_df.drop(columns=['_id'])
        
        # Rename the empty column to 'rec_num'
        if '' in filtered_df.columns:
            filtered_df = filtered_df.rename(columns={'': 'rec_num'})
        
        # Ensure we have data to return
        if filtered_df.empty:
            return df.to_dict('records')  # Return all data if filter returns empty
        
        return filtered_df.to_dict('records')
    except Exception as e:
        print(f"Error in update_dashboard: {e}")
        return df.to_dict('records')

@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")]
)
def update_graphs(viewData):
    """Create a pie chart showing breed distribution"""
    if viewData is None or len(viewData) == 0:
        return [html.P("No data available for chart")]
    
    try:
        dff = pd.DataFrame.from_dict(viewData)
        
        # Count breeds and get top 10 for better visualization
        breed_counts = dff['breed'].value_counts().head(10)
        
        if breed_counts.empty:
            return [html.P("No breed data available")]
        
        # Create pie chart
        fig = px.pie(
            values=breed_counts.values,
            names=breed_counts.index,
            title='Top 10 Breed Distribution'
        )
        
        fig.update_layout(
            height=400,
            showlegend=True,
            legend=dict(
                orientation="v",
                yanchor="top",
                y=1,
                xanchor="left",
                x=1.01
            )
        )
        
        return [dcc.Graph(figure=fig)]
    except Exception as e:
        return [html.P(f"Error creating chart: {str(e)}")]

@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    """Highlight selected columns in the data table"""
    if selected_columns is None:
        return []
    return [{
        'if': {'column_id': i},
        'background_color': '#D2F3FF'
    } for i in selected_columns]

@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")]
)
def update_map(viewData, index):
    """Update the geolocation map based on selected data"""
    if viewData is None or len(viewData) == 0:
        # Return default Austin map if no data
        return [
            dl.Map(
                style={'width': '100%', 'height': '400px'},
                center=[30.75, -97.48],
                zoom=10,
                children=[
                    dl.TileLayer(id="base-layer-id"),
                    dl.Marker(
                        position=[30.75, -97.48],
                        children=[
                            dl.Tooltip("Austin, TX"),
                            dl.Popup([html.H4("No animal selected")])
                        ]
                    )
                ]
            )
        ]
    
    try:
        dff = pd.DataFrame.from_dict(viewData)
        
        # Clean and validate location data
        dff = dff.dropna(subset=['location_lat', 'location_long'])
        
        if dff.empty:
            return [html.P("No location data available")]
        
        # Determine the row to highlight
        if index is not None and len(index) > 0 and index[0] < len(dff):
            row = index[0]
        else:
            row = 0
        
        # Get the selected animal's location
        selected_lat = dff.iloc[row]['location_lat']
        selected_long = dff.iloc[row]['location_long']
        selected_breed = dff.iloc[row]['breed']
        selected_name = dff.iloc[row]['name'] if dff.iloc[row]['name'] else "Unnamed"
        
        # Create markers for all animals in the filtered data
        markers = []
        for idx, animal in dff.iterrows():
            lat, lng = animal['location_lat'], animal['location_long']
            if pd.notna(lat) and pd.notna(lng):
                # Highlight the selected animal
                color = 'red' if idx == row else 'blue'
                markers.append(
                    dl.CircleMarker(
                        center=[lat, lng],
                        radius=5 if idx == row else 3,
                        color=color,
                        children=[
                            dl.Tooltip(f"{animal['breed']} - {animal['name'] if animal['name'] else 'Unnamed'}"),
                            dl.Popup([
                                html.H4(f"Animal: {animal['name'] if animal['name'] else 'Unnamed'}"),
                                html.P(f"Breed: {animal['breed']}"),
                                html.P(f"Age: {animal['age_upon_outcome']}"),
                                html.P(f"Sex: {animal['sex_upon_outcome']}")
                            ])
                        ]
                    )
                )
        
        return [
            dl.Map(
                style={'width': '100%', 'height': '400px'},
                center=[selected_lat, selected_long],
                zoom=12,
                children=[
                    dl.TileLayer(id="base-layer-id")
                ] + markers
            )
        ]
    except Exception as e:
        return [html.P(f"Error creating map: {str(e)}")]

print("The dashboard will be available at: http://127.0.0.1:8050/")

Loaded 10000 dog records from database
The dashboard will be available at: http://127.0.0.1:8050/


In [2]:
# Run the dashboard server
# The dashboard will open at http://127.0.0.1:8050/
# Press Ctrl+C in the output below to stop the server
app.run(debug=True, port=8050, jupyter_mode='external')

Dash app running on http://127.0.0.1:8050/
